#### Source of Datasets 
- Renewable energy usage percentage data preprocessed from BP (In preprocessed/usage/ directory)
    - processed.csv
    
    
- 1990-2018 GDP (https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.PCAP.CD&country=)
    - annual_gdp.csv
    - Downloaded as csv, then renamed file.
    
    
- Until 2018 Debt to GDP Percentage (https://www.gfmag.com/global-data/economic-data/public-debt-percentage-gdp)
    - debt_to_gdp.csv
    - Copied and pasted onto excel sheet, then converted to csv file.
    
    
- 2018 Renewable Energy Investment (Main Countries) (https://www.statista.com/statistics/799098/global-clean-energy-investment-by-country/)
    - renewable_investment_2018.csv
    - Downloaded as excel, removed excess words, added column names 'country' and 'usd_billions', and converted to csv file.
    
    
- 2016 Investment towards Developing Countries for Renewable Energy (https://www.sdg.org/datasets/174868e096e5417989316e64752b8d6d)
    - international_financial_flows_to_developing_countries.csv
    - Downloaded as excel spreadsheet, then converted to csv file.
    
    
- 2018 Number of Company Headquarters per country (https://www.naics.com/business-lists/counts-by-country/)
    - companies_count.csv
    - Copied and pasted onto excel, then converted to csv file.
    
    
- 2017 Industrial Production Growth Rate(https://www.cia.gov/library/publications/the-world-factbook/rankorder/2089rank.html)
    - industrial_production_growth_rate.txt
    - Downloaded the txt file and renamed the file.
    
    
- 1990-2018 Population (https://databank.worldbank.org/reports.aspx?source=2&series=SP.POP.TOTL&country=#)
    - population.csv
    - Downloaded as csv, and renamed the file.

In [1]:
import numpy as np
import pandas as pd

In [3]:
# Preprocessing of renewable energy ratio + gdp + population + debt dataset
# processed.csv + annual_gdp.csv + population.csv + debt_to_gdp.csv => renew_gdp_population.csv

renew_energy_ratio_df = pd.read_csv('../usage/processed.csv')
# renew_ratio does not include nuclear
renew_2017_df = renew_energy_ratio_df[renew_energy_ratio_df['year']==2017][['country', 'renew_energy_ratio']]
renew_2018_df = renew_energy_ratio_df[renew_energy_ratio_df['year']==2018][['country', 'renew_energy_ratio']]

annual_gdp_df = pd.read_csv('../../raw_data/finance/annual_gdp.csv')
annual_gdp_df = annual_gdp_df[['Country Name', '2017 [YR2017]','2018 [YR2018]']]
annual_gdp_df = annual_gdp_df.rename(columns={'Country Name':'country_name', '2017 [YR2017]':'gdp_2017', 
                                              '2018 [YR2018]':'gdp_2018'})
annual_gdp_df['gdp_2017'] = annual_gdp_df[(annual_gdp_df['gdp_2017']!='..')&(annual_gdp_df['gdp_2017']!='World')]['gdp_2017']
annual_gdp_df['gdp_2018'] = annual_gdp_df[(annual_gdp_df['gdp_2018']!='..')&(annual_gdp_df['gdp_2018']!='World')]['gdp_2018']
annual_gdp_df[['gdp_2017', 'gdp_2018']] = annual_gdp_df[['gdp_2017', 'gdp_2018']].astype(float)

population_df = pd.read_csv('../../raw_data/finance/population.csv')
population_df = population_df[['Country Name', '2017 [YR2017]', '2018 [YR2018]']]
population_df = population_df.rename(columns={'Country Name': 'country_name', '2017 [YR2017]':'population_2017', 
                                              '2018 [YR2018]': 'population_2018'})
population_df['population_2017'] = population_df[(population_df['population_2017']!='..')&(population_df['population_2017']!='World')]['population_2017']
population_df['population_2018'] = population_df[(population_df['population_2018']!='..')&(population_df['population_2018']!='World')]['population_2018']
population_df[['population_2017', 'population_2018']] = population_df[['population_2017', 'population_2018']].astype(float)

annual_gdp_population_df = annual_gdp_df.merge(population_df, left_on='country_name', right_on='country_name', 
                                               how='inner')
# No dropped countries

annual_gdp_population_df['gdp_per_capita_2017'] = annual_gdp_population_df['gdp_2017']/annual_gdp_population_df['population_2017']
annual_gdp_population_df['gdp_per_capita_2018'] = annual_gdp_population_df['gdp_2018']/annual_gdp_population_df['population_2018']
annual_gdp_population_df = annual_gdp_population_df

# Align names of countries
annual_gdp_population_df = annual_gdp_population_df.replace('Russian Federation', 'Russia')
annual_gdp_population_df = annual_gdp_population_df.replace('Venezuela, RB', 'Venezuela')
annual_gdp_population_df = annual_gdp_population_df.replace('Hong Kong SAR, China', 'Hong Kong')
annual_gdp_population_df = annual_gdp_population_df.replace('Egypt, Arab Rep.', 'Egypt')
annual_gdp_population_df = annual_gdp_population_df.replace('Korea, Dem. People’s Rep.', 'North Korea')
annual_gdp_population_df = annual_gdp_population_df.replace('Korea, Rep.', 'Korea')
annual_gdp_population_df = annual_gdp_population_df.replace('Iran, Islamic Rep.', 'Iran')
renew_2017_df = renew_2017_df.replace('Korea, Republic of', 'Korea')
renew_2017_df = renew_2017_df.replace('Viet Nam', 'Vietnam')
renew_2017_df = renew_2017_df.replace('Viet Nam', 'Vietnam')
renew_2017_df = renew_2017_df.replace('Iran (Islamic Republic of)', 'Iran')

renew_gdp_population_df = renew_2017_df.merge(annual_gdp_population_df, left_on='country', right_on='country_name', how='inner').drop('country_name', axis=1)
renew_gdp_population_df = renew_gdp_population_df.rename(columns={'renew_energy_ratio':'renew_ratio_2017'})

# Dropped Countries: {nan, 'French Polynesia', 'Haiti', 'Vanuatu', 'Upper middle income', 'Taiwan', 
# 'Pacific island small states', 'Grenada', 'Lebanon', 'Virgin Islands (U.S.)', 'North America', 
# 'Fragile and conflict affected situations', 'Burkina Faso', 'Malta', 'Eastern Africa', 'Other Northern Africa', 
# 'Guam', 'Maldives', 'Somalia', 'Mauritania', 'Yemen, Rep.', 'Europe & Central Asia', 'Sub-Saharan Africa', 
# 'Bermuda', 'St. Vincent and the Grenadines', 'Botswana', 'Gabon', 'Cuba', 'St. Kitts and Nevis', 
# 'West Bank and Gaza', 'Mauritius', 'Sudan', 'Mozambique', 'Tanzania', 'Madagascar', 'Kosovo', 'Sierra Leone', 
# 'Marshall Islands', 'Western Africa', 'Suriname', 'Senegal', 'Brunei Darussalam', 'Greenland', 'Nepal', 
# 'Liechtenstein', '                 European Union #', 'Mali', 'IDA only', 'Caribbean small states', 
# 'Sao Tome and Principe', 'Antigua and Barbuda', 'Sub-Saharan Africa (excluding high income)', 'Congo, Dem. Rep.',
# 'Lesotho', 'Georgia', 'Bahrain', 'Other Europe', 'Uganda', 'World', 'Post-demographic dividend', 'Angola', 
# 'Togo', 'Other small states', 'Tonga', 'Bhutan', 'Nicaragua', 'Europe & Central Asia (excluding high income)', 
# 'Curacao', 'New Caledonia', 'Belize', 'Liberia', 'Europe & Central Asia (IDA & IBRD countries)', 'Burundi', 
# 'Andorra', 'Middle East & North Africa (IDA & IBRD countries)', 'Low & middle income', 'Bolivia', 
# 'East Asia & Pacific', 'Comoros', 'Central Europe and the Baltics', 'Bosnia and Herzegovina', 'Paraguay', 
# 'Serbia', 'Cabo Verde', 'Timor-Leste', 'High income', 'Middle East & North Africa (excluding high income)', 
# 'Tunisia', 'Euro area', 'Croatia', 'Other CIS', 'Tuvalu', 'Ethiopia', 'Slovak Republic', 'Other Middle East', 
# 'Guatemala', 'St. Lucia', 'Central African Republic', 'Turks and Caicos Islands', 'Libya', 'Macao SAR, China', 
# 'San Marino', 'Guyana', 'Seychelles', 'South Sudan', 'Monaco', 'Northern Mariana Islands', 'Other South America',
# 'South Asia (IDA & IBRD)', 'British Virgin Islands', 'Benin', 'Jamaica', 'Costa Rica', 'Mongolia', 'Arab World', 
# 'Other Caribbean', 'Papua New Guinea', 'Sub-Saharan Africa (IDA & IBRD countries)', 'Lao PDR', 
# 'East Asia & Pacific (excluding high income)', 'Late-demographic dividend', "Cote d'Ivoire", 
# 'Least developed countries: UN classification', 'Dominican Republic', 'Niger', 'Solomon Islands', 'South Asia', 
# 'East Asia & Pacific (IDA & IBRD countries)', 'Panama', 'Malawi', 'Eritrea', 'Honduras', 'Tajikistan', 
# 'Latin America & Caribbean', 'Montenegro', 'Equatorial Guinea', 'OECD members', 'Faroe Islands', 'Gibraltar', 
# 'Samoa', 'North Macedonia', 'Pre-demographic dividend', 'American Samoa', 'Isle of Man', 'Guinea', 
# 'Guinea-Bissau', 'Slovakia', 'Latin America & Caribbean (excluding high income)', 
# Heavily indebted poor countries (HIPC)', 'Lower middle income', 'Kenya', 'Djibouti', 'Jordan', 'Small states', 
# 'Low income', 'El Salvador', 'Dominica', 'Myanmar', 'Kyrgyz Republic', 'Zimbabwe', 'Not classified', 'Rwanda', 
# 'Early-demographic dividend', 'IBRD only', 'Syrian Arab Republic', 'Gambia, The', 'Namibia', 'IDA total', 
# 'Aruba', 'Afghanistan', 'Congo, Rep.', 'Nigeria', 'Armenia', 'Other Asia Pacific', 'Barbados', 'Bahamas, The', 
# 'Ghana', 'St. Martin (French part)', 'European Union', 'Chad', 'Middle Africa', 'Albania', 'Fiji', 'Cameroon', 
# 'Cambodia', 'Kiribati', 'IDA blend', 'Eswatini', 'Middle East & North Africa', 'Channel Islands', 'Micronesia, 
# Fed. Sts.', 'Puerto Rico', 'Sint Maarten (Dutch part)', 'Palau', 'Moldova', 'Middle income', 'Central America', 
# 'North Korea', 'Latin America & the Caribbean (IDA & IBRD countries)', 'Uruguay', 'Nauru', 'IDA & IBRD total', 
# 'Zambia', 'Cayman Islands'}

# Align names of Countries
renew_2018_df = renew_2018_df.replace('Korea, Republic of', 'Korea')
renew_2018_df = renew_2018_df.replace('Viet Nam', 'Vietnam')
renew_2018_df = renew_2018_df.replace('Iran (Islamic Republic of)', 'Iran')

renew_gdp_population_df = renew_gdp_population_df.merge(renew_2018_df, left_on='country', right_on='country', how='inner')
renew_gdp_population_df = renew_gdp_population_df.rename(columns={'renew_energy_ratio':'renew_ratio_2018'})

# Dropped Countries :{'Taiwan', 'Other South America', 'Eastern Africa', 'Other Northern Africa', 
# 'Other Caribbean', 'Other Asia Pacific', 'Middle Africa', 'Western Africa', 
# '                 European Union #', 'Other CIS', 'Central America', 'Other Middle East', 'Slovakia', 
# 'Other Europe'}

debt_df = pd.read_csv('../../raw_data/finance/debt_to_gdp.csv')
debt_df = debt_df[['Country', '2017', '2018']]
debt_df = debt_df.rename(columns={'Country': 'country', '2017': 'debt_gdp_ratio_2017', '2018': 'debt_gdp_ratio_2018'})
debt_df[['debt_gdp_ratio_2017', 'debt_gdp_ratio_2018']] = debt_df[['debt_gdp_ratio_2017', 'debt_gdp_ratio_2018']]/100

renew_gdp_population_df = renew_gdp_population_df.merge(debt_df, left_on='country', right_on='country', how='inner')
renew_gdp_population_df.to_csv('../finance/renew_gdp_population.csv')

# Dropped Countries: {'Haiti', 'San Marino', 'Guyana', 'Vanuatu', 'Seychelles', 'South Sudan', 'Grenada', 
# 'Lebanon', 'Costa Roca', 'Burkina Faso', 'Malta', 'Benin', 'Jamaica', 'Maldives', 'Dominicana', 'Mauritania', 
# 'St. Vincent and the Grenadines', 'Taiwan Province of China', 'Botswana', 'Gabon', 'St. Kitts and Nevis', 
# 'Lao P.D.R.', 'Mauritius', 'Iran', 'Democratic Republic of the Congo', 'Dominican Republic', 'Niger', 'Sudan', 
# 'Mozambique', 'Tanzania', 'Madagascar', 'Kosovo', 'Solomon Islands', 'Sierra Leone', 'FYR Macedonia', 
# 'Marshall Islands', 'Panama', 'Malawi', 'Eritrea', 'Papa New Geinea', 'Honduras', 'Suriname', 'Bulgaria', 
# 'Senegal', 'Brunei Darussalam', 'Tajikistan', 'Montenegro', 'Equatorial Guinea', 'Nepal', 'Mali', 
# 'Sao Tome and Principe', 'Antigua and Barbuda', 'Samoa', 'Lesotho', 'Bulgaira', 'Guinea', 'Georgia', 
# 'Guinea-Bissau', 'Bahrain', 'Uganda', 'Angola', 'Togo', 'Kenya', 'Djibouti', 'Jordan', 'Hong Kong SAR', 
# 'Bhutan', 'Nicaragua', 'Islamic Republic of Iran', 'El Salvador', 'Myanmar', 'Belize', 'Kyrgyz Republic', 
# 'Zimbabwe', 'The Bahamas', 'Liberia', 'Rwanda', 'Republic of Congo', 'Burundi', "CUte d'Ivoire", 'Micronesia', 
# 'Aruba', 'Afghanistan', 'Nigeria', 'Barbados', 'Bolivia', 'Ghana', 'Comoros', 'Bosnia and Herzegovina', 'Chad', 
# 'The Gambia', 'Paraguay', 'Serbia', 'Cabo Verde', 'Albania', 'Fiji', 'Cameroon', 'Cambodia', 'Kiribati', 
# 'Timor-Leste', 'Eswatini', 'Puerto Rico', 'Tunisia', 'Croatia', 'Tuvalu', 'Moldova', 'Nambia', 'Ethiopia', 
# 'Slovak Republic', 'Guatemala', 'St. Lucia', 'Uruguay', 'Central African Republic', 'Nauru', 'Hong Kong', 
# 'Zambia', 'Yemen'}

In [4]:
# Preprocessing of renewable energy per country dataset
# renewable_investment_2018.csv + renew_gdp_population_df => invest_to_renewable.csv

invest_renew_df = pd.read_csv('../../raw_data/finance/renewable_investment_2018.csv')
invest_renew_df = invest_renew_df.merge(renew_gdp_population_df, left_on='country', right_on='country', how='inner') 
invest_renew_df = invest_renew_df.rename(columns={'usd_billions':'investment_2018'})

# Dropped Countries: {'Portugal', 'Romania', 'Indonesia', 'Azerbaijan', 'Czech Republic', 'Cyprus', 
# 'Turkmenistan', 'Estonia', 'Switzerland', 'Greece', 'Pakistan', 'Venezuela', 'Peru', 'New Zealand', 'Colombia', 
# 'Egypt', 'Saudi Arabia', 'Iceland', 'Kazakhstan', 'Thailand', 'Algeria', 'Austria', 'Uzbekistan', 'Philippines',
# 'Iraq', 'Ireland', 'Chile', 'Argentina', 'Ecuador', 'United Arab Emirates', 'Latvia', 'Singapore', 'Poland', 
# 'Kuwait', 'Finland', 'Brazil', 'Slovenia', 'Sri Lanka', 'Qatar', 'Bangladesh', 'Korea', 'Trinidad and Tobago', 
# 'Israel', 'Luxembourg', 'Hungary', 'Belarus', 'Russia', 'Oman', 'Malaysia', 'Lithuania'}

invest_renew_df['investment_gdp_ratio_2018'] = invest_renew_df['investment_2018']/invest_renew_df['gdp_2018']
invest_renew_df.to_csv('../finance/invest_to_renewable.csv')

In [5]:
# Preprocessing of investment towards developing countries dataset
# international_financial_flows_to_developing_countries.csv => invest_developing_countries.csv

invest_df = pd.read_csv('../../raw_data/finance/international_financial_flows_to_developing_countries.csv')
invest_df = invest_df[['geoAreaName', 'parentName', 'ISO3', 'value_2000', 'value_2001', 'value_2002',
       'value_2003', 'value_2004', 'value_2005', 'value_2006', 'value_2007',
       'value_2008', 'value_2009', 'value_2010', 'value_2011', 'value_2012',
       'value_2013', 'value_2014', 'value_2015', 'value_2016']]
invest_df = invest_df.rename(columns={'geoAreaName':'country', 'value_2000':'2000', 'value_2001':'2001', 'value_2002':'2002',
       'value_2003':'2003', 'value_2004':'2004', 'value_2005':'2005', 'value_2006':'2006', 'value_2007':'2007',
       'value_2008':'2008', 'value_2009':'2009', 'value_2010':'2010', 'value_2011':'2011', 'value_2012':'2012',
       'value_2013':'2013', 'value_2014':'2014', 'value_2015':'2015', 'value_2016':'2016'})
invest_df.to_csv('../finance/invest_developing_countries.csv')

In [6]:
# Preprocessing of number of businesses + renewable energy usage percentage dataset
# companies_count.csv + renew_gdp_population_df => num_business.csv

companies_df = pd.read_csv('../../raw_data/finance/companies_count.csv', usecols=['Country Name', 'Number of Businesses'])
companies_df = companies_df.rename(columns={'Country Name': 'country', 
                                            'Number of Businesses': 'num_business_2018'})
companies_df['num_business_2018'] = companies_df['num_business_2018'].str.replace(',', '')
companies_df['num_business_2018'] = companies_df['num_business_2018'].astype(int)

# Align names of countries
companies_df = companies_df.replace('England', 'United Kingdom')
companies_df = companies_df.replace('Usa', 'United States')
companies_df = companies_df.replace('Korea Rep Of', 'Korea')
companies_df = companies_df.replace('Trinidad & Tobago', 'Trinidad and Tobago')
companies_df = companies_df.replace('Russian Federation', 'Russia')

companies_full_df = companies_df.merge(renew_gdp_population_df, left_on='country', right_on='country', how='inner')
companies_full_df.to_csv('../finance/num_business.csv')

# Dropped Countries: {'Sudan', 'Maldives', 'Congo Democratic Rep', 'Botswana', 'Turks & Caicos Is', 'Micronesia Fed St', 
# 'Northern Mariana Is', 'Mali', 'Ivory Coast', 'Solomon Islands', 'Kyrgyzstan', 'Dominican Republic', 'Gabon', 
# 'Macedonia', 'Papua New Guinea', 'Zimbabwe', 'Uruguay', 'Tonga', 'Bosnia-herzegovina', 'Haiti', 'Niger', 
# 'Burundi', 'Bulgaria', 'Cook Islands', 'Seychelles', 'Aruba', 'Djibouti', 'Antigua & Barbuda', 'Grenada', 
# 'Anguilla', 'Sao Tome & Principe', 'Kenya', 'French Polynesia', 'Guam', 'Martinique', 'Virgin Islands Uk', 
# 'Eritrea', 'Wake Island', 'Scotland', 'Guinea-bissau', 'Gibraltar', 'Ghana', 'Andorra', 'Guadeloupe', 'Albania',
# 'Jordan', 'Vanuatu', 'Wales', 'Nicaragua', 'Montserrat', 'Taiwan', 'Fiji', 'Nauru', 'Armenia', 'Monaco', 
# 'Kiribati', 'Macao', 'Tajikistan', 'Nigeria', 'Mozambique', 'Somalia', 'Burkina Faso', 'Moldova', 'Congo', 
# 'French Guiana', 'Costa Rica', 'Falkland Islands', 'Central African Rep', 'Namibia', 'Paraguay', 'Zambia', 
# 'Cambodia', 'Panama', 'Guinea', 'Brunei', 'Bahrain', 'Barbados', 'Cayman Islands', 'Cameroon', 'Slovakia', 
# 'Sierra Leone', 'Comoros', 'Iran', 'Mauritius', 'Malta', 'Swaziland', 'Georgia', 'Bolivia', 'Bhutan', 
# 'St Kitts-nevis', 'Senegal', 'Syria', 'Jamaica', 'Guatemala', 'Chad', 'San Marino', 'Gambia', 'Honduras', 
# 'Benin', 'Bermuda', 'Yemen', 'Belize', 'Reunion', 'St Vincent', 'Ethiopia', 'Turkish Rep N Cyprus', 
# 'Afghanistan', 'Madagascar', 'Mauritania', 'El Salvador', 'Guyana', 'Dominica', 'Libya', 'Northern Ireland', 
# 'Bahamas', 'Myanmar', 'St Lucia', 'Equatorial Guinea', 'Lesotho', 'Cape Verde', 'Norfolk Island', 
# 'Faroe Islands', 'Rwanda', 'Laos', 'Greenland', 'Hong Kong', 'Liberia', 'St Helena', 'Croatia', 'Uganda', 
# 'Tuvalu', 'Togo', 'Suriname', 'Nepal', 'Liechtenstein', 'Tanzania', 'Tunisia', 'Angola', 'Lebanon', 'Malawi'}

In [7]:
# Preprocessing of industrial production growth rate + renewable energy usage percentage dataset
# industrial_production_growth_rate.txt + renew_gdp_population_df => industrial_production_growth_rate.csv

with open('../../raw_data/finance/industrial_production_growth_rate.txt') as f:
    ind_prod_list = f.readlines()
ind_prod_list = [row.split() for row in ind_prod_list]
for index, row in enumerate(ind_prod_list):
    ind_prod_list[index][0] = int(ind_prod_list[index][0])
    ind_prod_list[index][-1] = float(ind_prod_list[index][0])
    if len(ind_prod_list[index])>3:
        ind_prod_list[index][1] = ' '.join(ind_prod_list[index][1:-1])
        del ind_prod_list[index][2:-1]
ind_prod_df = pd.DataFrame(ind_prod_list).drop(0, axis=1)
ind_prod_df = ind_prod_df.rename(columns={1:'country', 2:'production_growth_rate_2017'})

# Align names of countries
ind_prod_df = ind_prod_df.replace('Korea, South', 'Korea')

ind_prod_full_df = ind_prod_df.merge(renew_gdp_population_df, left_on='country', right_on='country', how='inner')
ind_prod_full_df.to_csv('../finance/industrial_production_growth_rate.csv')

# Dropped Countries: {'Sudan', 'Maldives', 'Botswana', 'Mali', 'Solomon Islands', 'Kyrgyzstan', 
# 'Dominican Republic', 'Gabon', 'Macedonia', 'Papua New Guinea', 'Zimbabwe', 'Uruguay', 'Tonga', 'Haiti', 
# 'Niger', 'West Bank', 'Burundi', 'Bulgaria', 'Cook Islands', 'Seychelles', 'Djibouti', 
# 'Saint Vincent and the Grenadines', 'Grenada', 'Anguilla', 'Kenya', 'Eritrea', 'Ghana', 'Albania', 'Jordan', 
# 'Vanuatu', 'Nicaragua', 'Montserrat', 'Taiwan', 'Fiji', 'Armenia', 'Monaco', 'Kiribati', 'Timor-Leste', 
# 'Tajikistan', 'Nigeria', 'Somalia', 'Mozambique', 'Guinea-Bissau', 'Burkina Faso', 'Mongolia', 
# 'Congo, Democratic Republic of the', 'Moldova', 'Eswatini', 'Costa Rica', 'Namibia', 'Gaza Strip', 
# 'Paraguay', 'Zambia', 'Cambodia', 'Panama', 'Sao Tome and Principe', 'Guinea', 'Brunei', 'Bahrain', 
# 'Czech Republic', 'Cayman Islands', 'Barbados', 'Cameroon', 'Slovakia', 'Sierra Leone', 'Comoros', 'Iran', 
# 'Serbia', 'Congo, Republic of the', 'Mauritius', 'Malta', 'Cabo Verde', 'Antigua and Barbuda', 
# 'Turks and Caicos Islands', 'Georgia', 'Macau', 'Bolivia', 'Bhutan', 'Senegal', 'Syria', 'Czechia', 'Jamaica', 
# 'Guatemala', 'Chad', 'San Marino', 'Montenegro', 'New Caledonia', 'Honduras', 'Benin', 'Bermuda', 'Yemen', 
# 'Belize', 'Bahamas, The', 'Ethiopia', 'Kosovo', 'Afghanistan', 'Madagascar', 'Mauritania', 'Burma', 
# 'El Salvador', 'Guyana', 'Libya', 'Dominica', 'European Union', 'Lesotho', 'Equatorial Guinea', 'Faroe Islands',
# 'Rwanda', 'Laos', "Cote d'Ivoire", 'Hong Kong', 'Liberia', 'British Virgin Islands', 'Uganda', 'Croatia', 
# 'Tuvalu', 'Togo', 'Bosnia and Herzegovina', 'Suriname', 'Saint Kitts and Nevis', 'Nepal', 'Cuba', 'Puerto Rico', 
# 'Tanzania', 'Korea, North', 'Central African Republic', 'Tunisia', 'Angola', 'Samoa', 'Gambia, The', 'Lebanon', 
# 'Saint Lucia', 'Malawi'}